<a href="https://colab.research.google.com/github/iwatobipen/playground/blob/master/OpenFF_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install conda
!wget https://repo.continuum.io/miniconda/Miniconda3-4.4.10-Linux-x86_64.sh
!chmod ./Miniconda3-4.4.10-Linux-x86_64.sh
!time bash ./Miniconda3-4.4.10-Linux-x86_64.sh -b -f -p /usr/local
# install rdkit via conda command
!time conda install -c conda-forge rdkit -y

--2019-04-09 12:04:42--  https://repo.continuum.io/miniconda/Miniconda3-4.4.10-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58304693 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.4.10-Linux-x86_64.sh’

Miniconda3-4.4.10-L 100%[===================>]  55.60M   143MB/s    in 0.4s    

2019-04-09 12:04:43 (143 MB/s) - ‘Miniconda3-4.4.10-Linux-x86_64.sh’ saved [58304693/58304693]

chmod: missing operand after ‘./Miniconda3-4.4.10-Linux-x86_64.sh’
Try 'chmod --help' for more information.
PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: libf

In [2]:
import sys
import os
sys.path.append('/usr/local/lib/python3.6/site-packages/')
from rdkit import Chem
from rdkit import rdBase
print(rdBase.rdkitVersion)

2018.09.1


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [6]:
! conda config --add channels omnia --add channels conda-forge

In [7]:
! conda install -y openforcefield

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [0]:
from openforcefield.topology import Molecule
from openforcefield.topology import Topology
from openforcefield.utils import RDKitToolkitWrapper
from openforcefield.typing.engines.smirnoff import ForceField
from simtk import openmm
from simtk import unit


In [0]:

def get_energy(system, positions):
    """
    Return the potential energy.

    Parameters
    ----------
    system : simtk.openmm.System
        The system to check
    positions : simtk.unit.Quantity of dimension (natoms,3) with units of length
        The positions to use
    Returns
    ---------
    energy
    """

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
    return energy


In [0]:
mol = Molecule.from_smiles('CCO')

In [17]:
mol

Molecule with name '' and SMILES '[H][O][C]([H])([H])[C]([H])([H])[H]'

In [0]:
mol.generate_conformers()
positions = mol.conformers[0]

In [0]:
#load Force Field
ff = ForceField('smirnoff99Frosst.offxml')

In [0]:
topology = mol.to_topology()

In [26]:
orginal_system = ff.create_openmm_system(topology)


In AmberToolsToolkitwrapper.computer_partial_charges_am1bcc: Molecule '' has more than one conformer, but this function will only generate charges for the first one.


In [0]:
orig_energy = get_energy(orginal_system, positions)

In [29]:
print(orig_energy)

2.3399880921635527 kcal/mol


In [0]:
mol = Chem.MolFromSmiles('c1ccccc1')

In [0]:
rdw = RDKitToolkitWrapper()

In [0]:
ofmol = rdw.from_rdkit(mol)

In [0]:
ofmol.generate_conformers()

In [38]:
positions = ofmol.conformers[0]
topology = ofmol.to_topology()
orginal_system = ff.create_openmm_system(topology)
orig_energy = get_energy(orginal_system, positions)
print(orig_energy)

7.987402593200566 kcal/mol
